# Recycling pre-built deep learning models for extracting features

https://www.packtpub.com/mapt/book/all_books/9781787128422/3/ch03lvl1sec27/very-deep-convolutional-networks-for-large-scale-image-recognition

In [1]:
import numpy as np

In [2]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [3]:
# pre-built and pre-trained deep learning VGG16 model
base_model = VGG16(weights='imagenet', include_top=True)
for i, layer in enumerate(base_model.layers):
     print (i, layer.name, layer.output_shape)

553435136/553467096 [============================>.] - ETA: 0s0 input_1 (None, 224, 224, 3)
1 block1_conv1 (None, 224, 224, 64)
2 block1_conv2 (None, 224, 224, 64)
3 block1_pool (None, 112, 112, 64)
4 block2_conv1 (None, 112, 112, 128)
5 block2_conv2 (None, 112, 112, 128)
6 block2_pool (None, 56, 56, 128)
7 block3_conv1 (None, 56, 56, 256)
8 block3_conv2 (None, 56, 56, 256)
9 block3_conv3 (None, 56, 56, 256)
10 block3_pool (None, 28, 28, 256)
11 block4_conv1 (None, 28, 28, 512)
12 block4_conv2 (None, 28, 28, 512)
13 block4_conv3 (None, 28, 28, 512)
14 block4_pool (None, 14, 14, 512)
15 block5_conv1 (None, 14, 14, 512)
16 block5_conv2 (None, 14, 14, 512)
17 block5_conv3 (None, 14, 14, 512)
18 block5_pool (None, 7, 7, 512)
19 flatten (None, 25088)
20 fc1 (None, 4096)
21 fc2 (None, 4096)
22 predictions (None, 1000)


In [5]:
# extract features from block4_pool block
model = Model(input=base_model.input, output=base_model.get_layer('block4_pool').output)
img_path = 'cat.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

C:\Users\BNielson\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  


In [6]:
# get the features from this block
features = model.predict(x)

In [7]:
print(features)

[[[[   0.            0.          189.01783752 ...,    0.            0.
       0.        ]
   [   0.            0.            0.         ...,    0.           63.18912888
       0.        ]
   [   0.            0.            0.         ...,    0.          533.76947021
       0.        ]
   ..., 
   [   0.            0.            0.         ...,    0.          403.70797729
       0.        ]
   [   0.            0.           13.52689362 ...,    0.          374.7336731
       0.        ]
   [   0.            0.           76.14137268 ...,    0.            0.
       0.        ]]

  [[   0.            0.          162.07598877 ...,    0.            0.
       0.        ]
   [   0.          333.94525146    0.         ...,    0.            0.
       0.        ]
   [   0.            0.            0.         ...,    0.           31.43946457
       0.        ]
   ..., 
   [   0.            0.            0.         ...,    0.          572.50378418
       0.        ]
   [   0.            0.          

In [8]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 21s    


In [9]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)# let's add a fully-connected layer as first layer
x = Dense(1024, activation='relu')(x)# and a logistic layer with 200 classes as last layer
predictions = Dense(200, activation='softmax')(x)# model to train
model = Model(input=base_model.input, output=predictions)

C:\Users\BNielson\Anaconda3\envs\Keras\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  


In [10]:
# that is, freeze all convolutional InceptionV3 layers
for layer in base_model.layers: layer.trainable = False

In [11]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs model.fit_generator(...)

In [13]:
# we chose to train the top 2 inception blocks, that is, we will freeze

# the first 172 layers and unfreeze the rest: 
for layer in model.layers[:172]: layer.trainable = False 
for layer in model.layers[172:]: layer.trainable = True

In [16]:
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

In [18]:
# we train our model again (this time fine-tuning the top 2 inception blocks)
# alongside the top Dense layers
model.fit_generator()

TypeError: fit_generator() missing 2 required positional arguments: 'generator' and 'steps_per_epoch'